In [5]:
import os
import numpy as np
import mido
NUM_TRACKS = 3
NUM_TIMESHIFTS = 4
NUM_MIDI_PITCHES = 128

# Get note sequences for each track.

# Get final sequences of vectors.

In [6]:
def get_tempo(midi):
    """Get tempo from MidiFile object.

    Args:
       midi: MidiFile object.
       
    Returns:
        Tempo in microseconds.
    """
    tempo = -1
    for i, track in enumerate(midi.tracks):
        for msg in track:
            if msg.is_meta and msg.type == 'set_tempo':
                msg_str = str(msg)
                start = msg_str.index('tempo=')
                new_msg = msg_str[start + 6:]
                end = new_msg.index(' ')
                tempo = int(new_msg[:end])

    return tempo

def get_note_sequences(midi_file):
    """Get note sequences for each track from a MidiFile object.

    Args:
       midi_file: Filename.
       
    Returns:
        List of tracks containing note sequences, where each note is a dictionary containing the note type
        ('note_on' or 'note_off'), the MIDI pitch value, and duration of the note in beats.
    """
    midi = mido.MidiFile(midi_file)
    tracks = []

    for track in midi.tracks:
        note_sequences = []
        for msg in track:               
            if not msg.is_meta:
                note_vector = {}
                num_beats = msg.time / midi.ticks_per_beat 
                if msg.type == 'note_on' or msg.type == 'note_off':
                    # Get note value.
                    msg_str = str(msg)
                    start = msg_str.index('note=')
                    new_msg = msg_str[start + 5:]
                    end = new_msg.index(' ')
                    note = int(new_msg[:end])
                    #if (note == 79):
                    #    print(msg.time, midi.ticks_per_beat, num_beats)
                    
                    # Get velocity.
                    start = msg_str.index('velocity=')
                    new_msg = msg_str[start + 9:]
                    end = new_msg.index(' ')
                    velocity = int(new_msg[:end])

                    # Set note vector values.
                    note_vector["type"] = msg.type
                    if velocity == 0:
                        note_vector["type"] = 'note_off'
                    note_vector["note"] = note
                    note_vector["time"] = num_beats

                    # Add note vector to note sequence list.
                    note_sequences.append(note_vector)
        tracks.append(note_sequences)

    return tracks

In [7]:
def tracks_to_vector_sequence(tracks):
    """Get final vector from note sequence dictionaries.

    Args:
       tracks: List of tracks containing note sequence dictionaries with type (note_on or note_off),
       note, and time delay before note. Tracks are sorted from highest to lowest.
       
    Returns:
        Final one-hot vector containing NUM_MIDI_PITCHES * NUM_TRACKS note_on events,
        NUM_MIDI_PITCHES * NUM_TRACKS note_off events,
        and NUM_TIMESHIFTS timeshift events in intervals of 1/4 of a beat each.
    """
    if tracks == None:
        return []
    
    assert len(tracks) <= NUM_TRACKS + 1
    
    # Sort all events by start time.
    events = []
    for i, sequence in enumerate(tracks[1:]):
        start_time = 0
        for event in sequence:
            start_time += event["time"]
            # Round to nearest sixteenth note.
            rounded_start_time = np.round(start_time * NUM_TIMESHIFTS) / NUM_TIMESHIFTS
            new_event = {"type": event["type"], "track": i, "note": event["note"],
                         "start_time": rounded_start_time}
            events.append(new_event)
    events = sorted(events, key=lambda x: x["start_time"])
    
    # Fix time lengths.
    prev_start_time = 0
    for e in events:
        e["time"] = e["start_time"] - prev_start_time
        prev_start_time = e["start_time"]
    
    # Create final vector sequence.
    final_sequence = []
    for e in events:
        #time_vector = np.zeros(NUM_MIDI_PITCHES * 2 * NUM_TRACKS + NUM_TIMESHIFTS)
        time = e["time"]
        # Create rest vectors if time is greater than 1 beat.
        while time > 1.0:
            #rest_vector = np.zeros(NUM_MIDI_PITCHES * 2 * NUM_TRACKS + NUM_TIMESHIFTS)
            #rest_vector[-1] = 1
            #final_sequence.append(rest_vector)
            final_sequence.append(NUM_MIDI_PITCHES * 2 * NUM_TRACKS + NUM_TIMESHIFTS - 1)
            time -= 1.0
        if time > 0:
            timeshift = int(time * NUM_TIMESHIFTS) - 1
            #time_vector[NUM_MIDI_PITCHES * 2 * NUM_TRACKS + timeshift] = 1
            #final_sequence.append(time_vector)
            final_sequence.append(NUM_MIDI_PITCHES * 2 * NUM_TRACKS + timeshift)
        
        note_vector = np.zeros(NUM_MIDI_PITCHES * 2 * NUM_TRACKS + NUM_TIMESHIFTS)
        track_offset = NUM_MIDI_PITCHES * e["track"]
        # Set correct note.
        if e["type"] == 'note_on':
            #note_vector[track_offset + e["note"]] = 1
            final_sequence.append(track_offset + e["note"])
        else:
            #note_vector[NUM_MIDI_PITCHES * NUM_TRACKS + track_offset + e["note"]] = 1
            final_sequence.append(NUM_MIDI_PITCHES * NUM_TRACKS + track_offset + e["note"])
        #final_sequence.append(note_vector)
        
    return final_sequence

In [8]:
def beat_number_to_binary(n):
    """Get binary vector from number 0 to 15.

    Args:
       n: Integer from 0 to 15 inclusive.
       
    Returns:
        Length-4 vector representing n in binary. For example, n = 6, will return [1, 0, 1, 0].
    """
    return np.array([(n // 8) % 2, (n // 4) % 2, (n // 2) % 2, n % 2])

def get_beat_number_sequence(note_vector_sequence):
    """Get beat number sequence from note vector sequence.

    Args:
       note_vector_sequence: Final sequence of note and timeshift events.
       
    Returns:
        Returns sequence of beat number vectors, represented as numbers from 0 to 15.
    """
    current_beat = 0
    beat_numbers = []
    for event in note_vector_sequence:
        beat_numbers.append(current_beat)
        if event >= NUM_MIDI_PITCHES * NUM_TRACKS * 2:
            current_beat = (current_beat + event - NUM_MIDI_PITCHES * NUM_TRACKS * 2 + 1) % 16
    return beat_numbers

In [9]:
NUM_CHORDNOTES = 36
NUM_PITCHES = 12
def get_chordnote_sequence(note_vector_sequence, beat_sequence):
    """Get chord note sequences within in every beat.

    Args:
       note_vector_sequence: Final sequence of note and timeshift events.
       beat_sequence: Sequence of beat numbers.
       
    Returns:
        Returns sequence of notes for each track in each interval of length 1 beat.
    """
    chordnote_sequences = []
    
    chordnotes = np.zeros(NUM_CHORDNOTES)
    prev_b = 100
    for n, b in zip(note_vector_sequence, beat_sequence):
        if b//4 - prev_b//4 != 0 :
            chordnotes = np.zeros(NUM_CHORDNOTES)
        else:
            chordnotes = chordnote_sequences[-1].copy()
        if n < NUM_MIDI_PITCHES * NUM_TRACKS:
            track = n//NUM_MIDI_PITCHES
            chordnotes[n % NUM_PITCHES + NUM_PITCHES * track] = 1
        chordnote_sequences.append(chordnotes)
        prev_b = b
        
    return np.array(chordnote_sequences)

# Use the functions below to get midi to vector sequence.

In [10]:
def midi_to_vector(filename):
    """Get final vector from midi file.

    Args:
       filename: File name of midi.
       
    Returns:
        Final one-hot vector containing NUM_MIDI_PITCHES * NUM_TRACKS note_on events,
        NUM_MIDI_PITCHES * NUM_TRACKS note_off events,
        and NUM_TIMESHIFTS timeshift events in intervals of 1/4 of a beat each.
    """
    tracks = get_note_sequences(filename)
    return tracks_to_vector_sequence(tracks)

In [11]:
def get_training_data(start_index, end_index, n_input=100, training_filename='training_data.txt'):
    """Generate training data files for a subset of files in "midis_processed_nopercussion/" directory.
    
    Args:
       start_index: Start index of training data.
       end_index: End index of training data.
       n_input: Minimum length of sequence of notes.
       training_filename: Name of output text file that contains training data.
    """
    directory = 'midis_processed_nopercussion/'
    training_data_file = open(training_filename, 'w')
    beat_number_file = open('beat_number_' + training_filename, 'w')
    for i, filename in enumerate(os.listdir(directory)[start_index:end_index]):
        try:
            if filename.endswith(".mid"):
                vector = midi_to_vector(directory + filename)
                beats = get_beat_number_sequence(vector)
                assert len(vector) == len(beats)
                if len(vector) > n_input:
                    training_output_string = filename + '~'
                    beat_output_string = filename + '~'
                    for v, b in zip(vector, beats):
                        if v >= 772:
                            print("ERROR: " + str(v) + " " + filename)
                        training_output_string += ' ' + str(v)
                        beat_output_string += ' ' + str(b)
                    training_data_file.write(training_output_string + '\n')
                    beat_number_file.write(beat_output_string + '\n')
                else:
                    print("Training data too short: " + filename + ". len(vector)=" + str(len(vector)))
        except:
            print("Faulty training data: " + filename + ". len(vector)=" + str(len(vector)))
        if (i+1)%25 == 0:
            print(" > {}/{} files loaded.".format(i+1, len(os.listdir(directory)[start_index:end_index])))
               
    training_data_file.close()
    beat_number_file.close()

In [45]:
BIN_VECTOR = np.array([2**i for i in range(NUM_CHORDNOTES-1, -1, -1)])
def get_number_from_binary_vector(vector):
    """Get number from binary vector.

    Args:
       vector: Numpy array with binary representation.
       
    Returns:
        Integer.
    """
    return np.dot(vector, BIN_VECTOR)

def chordnote_to_binary(n):
    """Get binary vector from number 0 to 2**36-1.

    Args:
       n: Integer from 0 to 2**36-1 inclusive.
       
    Returns:
        Length-36 vector representing n in binary.
    """
    return (np.reshape(n, (-1, 1))//BIN_VECTOR.reshape((1, -1))) % 2

In [13]:
def write_chordnote_file(start_index, end_index, training_file, beat_file):
    """Generate chordnote file.
    
    Args:
       start_index: Start index of training data.
       end_index: End index of training data.
       training_file: Name of input text file that contains training data.
       beat_file: Name of input text file that contains beat data.
    """
    training_data, training_data_labels, beat_data = get_training_vectors_from_file(training_file, beat_file)
    directory = 'midis_processed_nopercussion/'
    chordnote_file = open('chordnote_training_data.txt', 'w')
    for i in range(start_index, end_index):
        try:
            vector = training_data[i]
            filename = training_data_labels[i]
            beats = beat_data[i]
            chordnotes = get_chordnote_sequence(vector, beats)
            output_string = filename + '~'
            for c in chordnotes:
                output_string += ' ' + str(get_number_from_binary_vector(c))
            chordnote_file.write(output_string + '\n')
        except:
            print(str(i) + ": Faulty training data: " + filename + ". len(vector)=" + str(len(vector)))
               
    chordnote_file.close()

In [14]:
def get_training_vectors_from_file(training_filename, beat_filename):
    """Generate vectors from training data text file.
    
    Args:
       training_filename: Name of text file that contains training data.
       beat_filename: Name of beat numbers text file.
       
    Returns:
        Numpy arrays of training data, training data labels, and beat numbers.
    """
    training_data_file = open(training_filename, 'r')
    beat_file = open(beat_filename, 'r')
    training_data = []
    training_data_labels = []
    beat_numbers = []
    for line in training_data_file:
        index = line.index('~')
        filename = line[:index]
        tokens = line[index + 2:].strip().split(' ')
        vector = np.array(list(map(int, tokens)))
        for v in vector:
            if v > 772:
                print("ERROR: " + str(v) + ' ' + filename)
        training_data.append(np.array(vector))
        training_data_labels.append(filename)
        
    for line in beat_file:
        index = line.index('~')
        filename = line[:index]
        tokens = line[index + 2:].strip().split(' ')
        vector = np.array(list(map(int, tokens)))
        for v in vector:
            if v >= 16 or v < 0:
                print("ERROR: " + str(v) + ' ' + filename)
        beat_numbers.append(np.array(vector))
        
    return np.array(training_data), training_data_labels, beat_numbers

In [15]:
def get_chordnote_sequence_from_file(chordnote_filename):
    """Generate vectors from chord notes text file.
    
    Args:
       chordnote_filename: Name of text file that contains chordnote data.
       
    Returns:
        Numpy arrays of chordnote data.
    """
    chordnote_file = open(chordnote_filename, 'r')
    chordnotes = []
        
    for line in chordnote_file:
        index = line.index('~')
        filename = line[:index]
        tokens = line[index + 2:].strip().split(' ')
        vector = np.array(list(map(int, tokens)))
        chordnotes.append(np.array(vector))
        
    return chordnotes